In [ ]:
import logging
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
import sys
import time
import torch
import torch.nn as nn

from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from torch.optim import Adam
from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST
from torchvision.datasets import CIFAR10
from torchvision.datasets import CIFAR100
from torchvision.datasets import SVHN
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# !rm -rf /content/backprop/

In [ ]:
ff_type = 'backprop'

dataset = 'MNIST'
#dataset = 'FashionMNIST'
#dataset = 'CIFAR10'
#dataset = 'CIFAR100'
# dataset = 'SVHN'

In [ ]:
if not os.path.exists(ff_type):
  os.mkdir(ff_type)
        
path = os.path.join(ff_type, dataset)
if not os.path.exists(path):
  os.mkdir(path)

In [ ]:
def start_logging(logger):
    print("******* nvidia-smi logging started *******")
    while True:
        nvidia_smi_entry = os.popen("nvidia-smi --query-gpu=timestamp,temperature.gpu,utilization.gpu,utilization.memory,memory.used,memory.free,memory.total,power.draw --format=csv | sed -n '2 p'").read()
        logger.info(nvidia_smi_entry)
    print("******* nvidia-smi logging ended *******")

In [ ]:
formatter = logging.Formatter('%(message)s')
def setup_logger(name, log_file, level=logging.INFO):
    """To setup as many loggers as you want"""

    handler = logging.FileHandler(log_file)
    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.addHandler(handler)
    
    return logger

In [ ]:
# nvidia smi logger

logger_nvidia_smi = setup_logger(path+"_logger_nvidia_smi", os.path.join(path, "nvidia_smi.csv"))
system_logger = multiprocessing.Process(target=start_logging, args=[logger_nvidia_smi])

# layer loggers

logger_layer_ff = setup_logger(path+"_logger_layer_ff", os.path.join(path, "layer_ff.csv"))
logger_layer_ff.info('layer_num,layertime_ms')

# e2e loggers

logger_e2e_ff = setup_logger(path+"_logger_e2e_ff", os.path.join(path, "e2e_ff.csv"))
logger_e2e_ff.info('e2etime_ms')

# linear ff loggers

logger_gpu_compute_linear_ff = setup_logger(path+"_logger_gpu_compute_linear_ff", os.path.join(path, "gpu_compute_linear_ff.csv"))
logger_gpu_compute_linear_ff.info('epoch,gpucomputetime_ms')

logger_epoch_linear_ff = setup_logger(path+"_logger_epoch_linear_ff", os.path.join(path, "epoch_linear_ff.csv"))
logger_epoch_linear_ff.info('epoch_num,epochtime_ms')

INFO:backprop/SVHN_logger_layer_ff:layer_num,layertime_ms
INFO:backprop/SVHN_logger_e2e_ff:e2etime_ms
INFO:backprop/SVHN_logger_gpu_compute_linear_ff:epoch,gpucomputetime_ms
INFO:backprop/SVHN_logger_epoch_linear_ff:epoch_num,epochtime_ms


In [ ]:
# layer ff timers

layer_ff_start = torch.cuda.Event(enable_timing=True)
layer_ff_end = torch.cuda.Event(enable_timing=True)

# e2e ff timers

e2e_ff_start = torch.cuda.Event(enable_timing=True)
e2e_ff_end = torch.cuda.Event(enable_timing=True)

# linear ff timers

gpu_compute_linear_ff_start = torch.cuda.Event(enable_timing=True)
gpu_compute_linear_ff_end = torch.cuda.Event(enable_timing=True)
epoch_linear_ff_start = torch.cuda.Event(enable_timing=True)
epoch_linear_ff_end = torch.cuda.Event(enable_timing=True)

In [ ]:
def MNIST_loaders(train_batch_size=256, test_batch_size=256):

    transform = Compose([
        ToTensor(),
        Normalize((0.1307,), (0.3081,)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = DataLoader(
        MNIST('./data/', train=True,
              download=True,
              transform=transform),
        batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(
        MNIST('./data/', train=False,
              download=True,
              transform=transform),
        batch_size=test_batch_size, shuffle=False)

    return train_loader, test_loader

In [ ]:
def FashionMNIST_loaders(train_batch_size=256, test_batch_size=256):

    transform = Compose([
        ToTensor(),
        Normalize((0.1307,), (0.3081,)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = DataLoader(
        FashionMNIST('./data/', train=True,
              download=True,
              transform=transform),
        batch_size=train_batch_size, shuffle=True)

    test_loader = DataLoader(
        FashionMNIST('./data/', train=False,
              download=True,
              transform=transform),
        batch_size=test_batch_size, shuffle=False)

    return train_loader, test_loader

# train_loader, test_loader = FashionMNIST_loaders()

In [ ]:
def CIFAR10_loaders(train_batch_size=256, test_batch_size=256):

    transform = Compose([
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.4914, 0.4822, 0.4465)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = DataLoader(
        CIFAR10('./data/', train=True,
              download=True,
              transform=transform),
        batch_size=train_batch_size, shuffle=True)

    test_loader = DataLoader(
        CIFAR10('./data/', train=False,
              download=True,
              transform=transform),
        batch_size=test_batch_size, shuffle=False)

    return train_loader, test_loader

# train_loader, test_loader = CIFAR10_loaders()

In [ ]:
def CIFAR100_loaders(train_batch_size=256, test_batch_size=256):

    transform = Compose([
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = DataLoader(
        CIFAR100('./data/', train=True,
              download=True,
              transform=transform),
        batch_size=train_batch_size, shuffle=True)

    test_loader = DataLoader(
        CIFAR100('./data/', train=False,
              download=True,
              transform=transform),
        batch_size=test_batch_size, shuffle=False)

    return train_loader, test_loader

# train_loader, test_loader = CIFAR100_loaders()

In [ ]:
def SVHN_loaders(train_batch_size=256, test_batch_size=256):

    transform = Compose([
        ToTensor(),
        Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = DataLoader(
        SVHN('./data/', split='train',
              download=True,
              transform=transform),
        batch_size=train_batch_size, shuffle=True)

    test_loader = DataLoader(
        SVHN('./data/', split='test',
              download=True,
              transform=transform),
        batch_size=test_batch_size, shuffle=False)

    return train_loader, test_loader

# train_loader, test_loader = SVHN_loaders()

In [ ]:
class Net(nn.Module):
    
    def __init__(self, dims, num_labels):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(dims[0], dims[1])
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(dims[1], dims[2])
        self.linear3 = nn.Linear(dims[2], num_labels)
    
    def forward(self,x):
        out1 = self.linear1(x)
        act1 = self.relu(out1)
        out2 = self.linear2(act1)
        act2 = self.relu(out2)
        out3 = self.linear3(act2)
        return out3

In [ ]:
net = Net([28*28, 250, 110], num_labels=10)
net

Net(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=250, out_features=110, bias=True)
  (linear3): Linear(in_features=110, out_features=10, bias=True)
)

In [ ]:
def train(model, criterion, train_loader, test_loader, optimizer, epochs, input_size):
    for epoch in range(epochs):
        epoch_linear_ff_start.record() # start recording epoch time
        print(f"Epoch {epoch} of {epochs}")
        train_correct = 0
        total_train_loss = 0
        for i, data in enumerate(tqdm(train_loader)):
            gpu_compute_linear_ff_start.record() # start recording gpu compute time
            image, label = data
            image, label = image.to(device), label.to(device)
            optimizer.zero_grad()
            
            z = model(image.view(-1, input_size))
            _, prediction = torch.max(z, 1)
            loss = criterion(z, label)
            total_train_loss +=loss.item()
            loss.backward()
            optimizer.step()
            train_correct += (prediction == label).sum().item()
            gpu_compute_linear_ff_end.record() # end recording gpu compute time
            torch.cuda.synchronize()
            logger_gpu_compute_linear_ff.info(str(epoch)+","+str(gpu_compute_linear_ff_start.elapsed_time(gpu_compute_linear_ff_end))) # log gpu compute time
        epoch_linear_ff_end.record() # end recording epoch time
        torch.cuda.synchronize()
        logger_epoch_linear_ff.info(str(epoch)+","+str(epoch_linear_ff_start.elapsed_time(epoch_linear_ff_end))) # log epoch time
        train_error = 100 * (1-(train_correct / len(train_loader.dataset)))
        train_accuracy = 100 * (train_correct / len(train_loader.dataset))
        print('\nTraining Loss {0}, Accuracy {1:.2f}, Error: {2:.2f}%'.format(total_train_loss/len(train_loader.dataset), train_accuracy, train_error))
    

    test_correct = 0
    total_test_loss = 0
    for i, data in enumerate(test_loader):
        image, label = data 
        image, label = image.to(device), label.to(device)
        z = model(image.view(-1, input_size))
        _, prediction = torch.max(z, 1)
        test_correct += (prediction == label).sum().item()
        loss = criterion(z, label)
        total_test_loss += loss.item()

    test_error = 100 * (1-(test_correct / len(test_loader.dataset)))
    test_accuracy = 100 * (test_correct / len(test_loader.dataset))
    print('\nTest Loss {0}, Accuracy {1:.2f}, Error: {2:.2f}%'.format(total_test_loss/len(test_loader.dataset), test_accuracy,test_error))
    
    return train_error, test_error

In [ ]:

if __name__ == "__main__":
    reference_time = time.time()

    torch.manual_seed(1234)

    if dataset == 'MNIST':
      train_loader, test_loader = MNIST_loaders()
    elif dataset == 'FashionMNIST':
      train_loader, test_loader = FashionMNIST_loaders()
    elif dataset == 'CIFAR10': 
      train_loader, test_loader = CIFAR10_loaders()
    elif dataset == 'CIFAR100':
      train_loader, test_loader = CIFAR100_loaders()
    elif dataset == 'SVHN':
      train_loader, test_loader = SVHN_loaders()
    else:
      print('Invalid dataset')
      exit()
    
    if dataset == 'MNIST' or dataset == 'FashionMNIST':
      input_size = 784
    elif dataset == 'SVHN' or dataset == 'CIFAR10' or dataset == 'CIFAR100':
      input_size = 3072
    
    if dataset == 'MNIST' or dataset == 'FashionMNIST':
      model = Net([input_size, 250, 110], num_labels=10).to(device) # MNIST and FashionMNIST
      epochs = 20
    elif dataset == 'CIFAR10':
      model = Net([input_size, 250, 110], num_labels=10).to(device) # CIFAR10, CIFAR100
      epochs = 20
    elif dataset == 'CIFAR100':
      model = Net([input_size, 250, 110], num_labels=100).to(device)
      epochs = 20
    elif dataset == 'SVHN':
      model = Net([input_size, 75, 50], num_labels=10).to(device) # SVHN
      epochs = 20
    else:
      print('Invalid dataset')
      exit()
    
    # epochs = 100

    # model = Net([input_size, 500, 500]).to(device) # MNIST and FashionMNIST
    criterion = nn.CrossEntropyLoss()
    learning_rate = 0.00067
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    system_logger.start()
    e2e_ff_start.record() # start recording e2e train time
    train_error, test_error = train(model, criterion, train_loader, test_loader, optimizer, epochs, input_size)
    e2e_ff_end.record() # end recording e2e train time
    system_logger.terminate()
    torch.cuda.synchronize()
    logger_e2e_ff.info(str(e2e_ff_start.elapsed_time(e2e_ff_end))) # log e2e train time
    # train_error, test_error = train(model, criterion, train_loader, test_loader, optimizer, epochs, input_size, logger_gpu_compute, logger_e2e, reference_time)

    # elif dataset == 'CIFAR10' or dataset == 'CIFAR100':
    #   net = Net([3072, 500, 500]) # CIFAR10, CIFAR100
    # elif dataset == 'SVHN':
    #   net = Net([3072, 100, 100]) # SVHN
    # else:
    #   print('Invalid dataset')
    #   exit()

100%|██████████| 182040794/182040794 [00:05<00:00, 31476384.48it/s]


100%|██████████| 64275384/64275384 [00:03<00:00, 21373929.07it/s]


Epoch 0 of 20


  0%|          | 0/287 [00:00<?, ?it/s]

******* nvidia-smi logging started *******


...


Training Loss 0.0017963182523101945, Accuracy 86.30, Error: 13.70%
Epoch 19 of 20


  0%|          | 0/287 [00:00<?, ?it/s]INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:18:23.832, 58, 1 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.40 W

INFO:backprop/SVHN_logger_gpu_compute_linear_ff:19,8.23363208770752
  0%|          | 1/287 [00:00<01:06,  4.28it/s]INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:18:24.031, 58, 1 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_gpu_compute_linear_ff:19,9.200096130371094
  1%|          | 2/287 [00:00<01:01,  4.64it/s]INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:18:24.237, 58, 1 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_gpu_compute_linear_ff:19,3.338047981262207
  1%|          | 3/287 [00:00<01:07,  4.19it/s]INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:18:24.444, 58, 0 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_gpu_compute_linear_ff:19,4.057983875274658
  1%|▏         | 4/287 [00:00<01:04,  4.37it/s]INFO:backprop/SVHN_logger_nvidia_s


Training Loss 0.001782040691805984, Accuracy 86.38, Error: 13.62%


INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:19:15.639, 58, 1 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:19:15.749, 58, 0 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:19:15.846, 58, 0 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:19:15.962, 58, 0 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:19:16.075, 58, 0 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:19:16.208, 58, 0 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.59 W

INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:19:16.300, 58, 0 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:19:16.396, 58, 0 %, 0 %, 847 MiB, 14254 MiB, 15360 MiB, 29.50 W

INFO:backprop/SVHN_logger_nvidia_smi:2023/04/20 02:19:16.507, 58, 0 %, 0


Test Loss 0.002927319870800348, Accuracy 80.52, Error: 19.48%


In [ ]:
!zip -r /content/backprop.zip /content/backprop/

updating: content/backprop/ (stored 0%)
updating: content/backprop/MNIST/ (stored 0%)
updating: content/backprop/MNIST/epoch_linear_ff.csv (deflated 44%)
updating: content/backprop/MNIST/nvidia_smi.csv (deflated 93%)
updating: content/backprop/MNIST/gpu_compute_linear_ff.csv (deflated 58%)
updating: content/backprop/MNIST/layer_ff.csv (deflated 4%)
updating: content/backprop/MNIST/e2e_ff.csv (stored 0%)
updating: content/backprop/FashionMNIST/ (stored 0%)
updating: content/backprop/FashionMNIST/epoch_linear_ff.csv (deflated 45%)
updating: content/backprop/FashionMNIST/nvidia_smi.csv (deflated 93%)
updating: content/backprop/FashionMNIST/gpu_compute_linear_ff.csv (deflated 58%)
updating: content/backprop/FashionMNIST/layer_ff.csv (deflated 4%)
updating: content/backprop/FashionMNIST/e2e_ff.csv (stored 0%)
updating: content/backprop/CIFAR10/ (stored 0%)
updating: content/backprop/CIFAR10/epoch_linear_ff.csv (deflated 46%)
updating: content/backprop/CIFAR10/nvidia_smi.csv (deflated 93%)
u